In [1]:
from __future__ import print_function
import collections
import numpy as np
import tensorflow as tf
import numpy as np
import zipfile
import pandas as pd
import os
import shutil
import http
import matplotlib.pyplot as plt
from scipy import ndimage
from multiprocessing import Pool, Process
import threading
from six.moves import cPickle as pickle
from six.moves.urllib.request import urlretrieve
from six.moves.urllib.parse import quote
from sklearn import svm
from sklearn import cross_validation

%matplotlib inline

In [2]:
csv = pd.read_csv('children_de_DE.csv', index_col="imageId")
csv.drop('title', axis=1, inplace=True)
csv.drop('pageURL', axis=1, inplace=True)
#csv = csv[csv["breadCrumbs"].notnull()]
csv["imageURL"] = csv["imageURL"] + "&fit=inside|128:128"
csv["imageURL"] = csv["imageURL"].str.split(pat="http://cdn.home24.net", expand=True)[1]
#csv["breadCrumbs"] = csv["breadCrumbs"].str.split("|").str[-1].astype(int)
#csv[:10]
#csv["imageURL"]

In [103]:

#quote("/resized/media/catalog/product/B/r/Breite-45cmbrTiefe-44cmbrHöhe-201cm-932626.jpg?output-format=jpg&interpolation=progressive-bicubic&fit=inside|128:128")

csv

,imageURL,sku,breadCrumbs
imageId,,,
9dbb6c1aa94dc92e5e99ff19bccaf5b2,/resized/media/catalog/product/n/a/nachtschran...,M-AM-STI-3394,161718
c76bcbac1d1a240dba03d12c68a8cb5a,/resized/media/catalog/product/a/r/arte-m-nako...,M-AM-STI-3394,161718
f226f03b964803e6d3a6aa6a9da52d5e,/resized/media/catalog/product/n/a/nachtschran...,M-AM-STI-3394,161718
81e8f99a0527f8e8064a8d79fd13f1e1,/resized/media/catalog/product/n/a/nachtschran...,M-AM-STI-3394,161718
36f419c03527736d393f66be8d3fb0d0,/resized/media/catalog/product/n/a/nachtschran...,M-AM-STI-3394,161718
201fbfed8e906476dcd0e3bfa76caaf3,/resized/media/catalog/product/n/a/nachtschran...,M-AM-STI-3394,161718
5eaff3b15339ab03e26ed4299383f574,/resized/media/catalog/product/f/r/fredriks-ch...,M-AC-ATI-0190,161902
1dd9d5e56708a0ac8f45f045435da63d,/resized/media/catalog/product/f/r/fredriks-ch...,M-AC-ATI-0190,161902
e68f67c09c9ceaac34c8c7b0aab1b6c9,/resized/media/catalog/product/c/o/couchtisch-...,M-AC-WTI-0131,161314


In [152]:
def maybe_load(url, h, conn=None, force=False):
    if conn==None:
        conn = http.client.HTTPConnection("cdn.home24.net:80")
    if not os.path.exists("images"):
        os.mkdir("images")
    
    filename = get_name(h)
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    if force or not os.path.exists(filename):
        try:
            conn.request("GET", quote(url))
            res = conn.getresponse()
            data = res.read()
            if res.status == 404:
                return False
            with open(filename, "wb") as f:
                f.write(data)
            image = ndimage.imread(filename).astype(int)
            print("ok", h)
        except Exception as e:
            print("err", h, e, url)
            if os.path.exists(filename):
                os.remove(filename)
        
def _maybe_load(arr):
    maybe_load(arr[0], arr[2])
def get_name(h):
    return os.path.join("images", k[:2], k+".jpg")

def _process_q(q):
    for p in q:
        p.start()
    for p in q:
        p.join()

# with Pool(8) as p:
#     p.map(_maybe_load, data)

i = 1
conn = [ http.client.HTTPConnection("cdn.home24.net:80") for i in range(30)]
q = []
for k, img in csv.iterrows():
    p = threading.Thread(target=maybe_load, args=(img["imageURL"], k, conn[len(q)]))
    q.append(p)
    if len(q) >= 25:
        _process_q(q)
        q = []
    
    if (i%100==0):
        print(i)
    i=i+1
print(i)
_import_q(q)

# for k, img in csv.iterrows():
#     image_file = get_name(k)
#     new_file = os.path.join("images", k[:2], k+".jpg")
#     if os.path.exists(image_file):
#         dirname = os.path.dirname(new_file)
#         if not os.path.exists(dirname):
#             os.mkdir(dirname)
#         shutil.move(image_file, new_file)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [81]:
csv.loc["a6c6cb35922b3ff8626d9e1eb53e1f9b"]["breadCrumbs"]

'156142|156290|157486'

In [3]:
def get_name(h):
    return os.path.join("images", k[:2], k+".jpg")

In [4]:
image_size=128
folder="images_processed"

def maybe_convert(hname, session):
    image_file  = get_name(hname)
    result_file = os.path.join(folder, hname+".jpg")
    if os.path.exists(result_file):
        return True
    if not os.path.exists(folder):
        os.mkdir(folder)
    try:
        image = ndimage.imread(image_file).astype(int)
    except IOError as e:
        return False
    if image.shape == (128, 128, 3):
        shutil.copy2(image_file, result_file)
        return True
    print("convert ", image_file, image.shape)
    resized = tf.image.resize_image_with_crop_or_pad(image, 128, 128)
    resized = tf.cast(resized, tf.uint8)
    resized_bytes = tf.image.encode_jpeg(resized)
    
    with open(result_file, 'wb') as f:
        f.write(session.run(resized_bytes))
    return True

processed_res = list()
with tf.Session() as session:
#     with Pool(8) as p:
#         p.starmap(maybe_convert, [(h, session) for h in data[:3, 2]])
    i=0
    q = []
    for k, img in csv.iterrows():
        # pass
        i = i+1
        res = maybe_convert(k, session)
        processed_res.append(res)
        if (i%1000==0):
            print(i)
    print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [7]:
def apply(x):
    return x[-1]
    #return x[2] if len(x) > 3 else x[-1]
csv_filter = csv[processed_res]
csv_filter = csv_filter[csv_filter["breadCrumbs"].notnull()]
csv_filter["breadCrumbs"] = csv_filter["breadCrumbs"].str.split("|").apply(apply).astype(int)

In [8]:
csv_filter["breadCrumbs"].value_counts()

161246    3335
161342    1906
177757    1546
161686    1387
161358    1232
165350    1122
161702    1020
161314     979
161270     977
177597     903
177601     825
161478     775
161350     755
165342     736
161902     611
177877     599
172238     594
161338     587
165574     583
177753     575
161442     560
161310     557
161346     522
162310     505
165358     496
161518     488
177749     469
161250     452
161718     450
161454     433
          ... 
172222       2
161838       1
170686       1
162946       1
162466       1
177673       1
162414       1
161474       1
156218       1
162962       1
161802       1
165686       1
156330       1
162522       1
157546       1
165702       1
162138       1
177793       1
177573       1
171146       1
156770       1
178037       1
156542       1
157526       1
156222       1
156902       1
177785       1
156142       1
162418       1
165546       1
Name: breadCrumbs, dtype: int64

In [9]:
#csv["breadCrumbs"].value_counts()[:50]
# 161702 schlafzimmermoebel/kleiderschraenke/schwebetuerenschraenke/
# 177757 Boxspringbetten
# 
small_csv = csv_filter[csv_filter["breadCrumbs"].isin([161702, 177757])]
#X_train, X_test, y_train, y_test = 
#    cross_validation.train_test_split(X, y, test_size=0.1, random_state=42)

In [10]:
small_csv["breadCrumbs"].value_counts()

177757    1546
161702    1020
Name: breadCrumbs, dtype: int64

In [13]:
pixel_depth = 255.0

# img_file = "images_processed/05beb657f2280c3d5a2b455e0f3e73ca.jpg"
# rgb = ndimage.imread(img_file).astype(int)
# rgb = (rgb - pixel_depth/2) / pixel_depth
# rgb.shape

In [12]:


def read_gray(filename):
    #https://en.wikipedia.org/wiki/Grayscale#Converting_color_to_grayscale
    rgb = read_img(filename)
    #gray = np.dot(rgb[:,:,:3], [0.299, 0.587, 0.114])
    gray = np.dot(rgb[:,:,:3], [0.33333, 0.33334, 0.33333])
    
    #print(rgb.shape)
    #print(gray.shape)
    #plt.imshow(gray)
    return gray

def read_img(filename):
    rgb = ndimage.imread(filename).astype(float)
    rgb = (rgb - pixel_depth/2) / pixel_depth
    return rgb

def load_dataset(csv):
    dataset = np.ndarray(shape=(len(csv), image_size, image_size, 3),
                         dtype=np.float32)
    labels = np.ndarray(shape=(len(csv),), dtype=np.int32)
    i = -1
    for h, row in csv.iterrows():
        i = i+1
        image_file = os.path.join("images_processed", h+".jpg")
        try:
            img = read_img(image_file)
            
            if img.shape != (image_size, image_size, 3):
                raise Exception('Unexpected image shape: %s [%s]' % (str(image_data.shape), image_file))
            dataset[i] = img 
            labels[i] = csv.loc[h]["breadCrumbs"]
        except IOError as e:
            print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
            
    print('Full dataset tensor:', dataset.shape)
    return (dataset, labels)


def save_pickle(data, filename):
    try:
        bytes_out = pickle.dumps(data)
        print(type(bytes_out), len(bytes_out), 1<<10)
        with open(filename, 'wb') as f:
            f.write()
#             max_bytes = 1024
#             for idx in range(0, len(bytes_out), 1024):
#                 f.write(bytes_out[idx:idx+max_bytes])
        print("Saved")
    except Exception as e:
        print('Unable to save data to', filename, ':', e)
        

d, l = load_dataset(small_csv)
fidx = [l>0]
save_pickle((d[fidx],l[fidx]), "processed_h24.pickle")
#print(read_gray(folder+"/9dbb6c1aa94dc92e5e99ff19bccaf5b2.png"))
    
    

NameError: name 'pixel_depth' is not defined

## Processed images

In [111]:
data = pickle.load(open('processed_h24.pickle', 'rb'))

In [112]:
X = np.reshape(np.array(data[0]), (len(data[0]), 128*128))

In [118]:
X[0]

10689

In [114]:
i = 0
y = np.ndarray(shape=(len(data[1])), dtype=np.int64)
labels = dict()
accos = dict()
uniq_idx = 0
for i, sku in enumerate(data[1]):
    if sku not in labels:
        labels[sku] = uniq_idx
        accos[uniq_idx] = sku
        uniq_idx = uniq_idx + 1
    y[i] = labels[sku]

num_labels = len(labels)
one_hot_labels = (np.arange(num_labels) == y[:,None]).astype(np.float64)

In [115]:
del(data)

In [67]:
model = svm.SVC(probability=True)
model.fit(X[:100], y[:100])


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
scores = model.predict_proba(X[10000:10002])

print(scores[0].argsort()[:10])
print(scores[1].argsort()[:10])

[33  1 21  2 27 58 91 94 62 23]
[83  7 68 18 77  3 56 40 71  5]


In [69]:
print(y[10000])
print(accos[178])
print(accos[358])
print(accos[212])

178
M-MF-KL-07-24JO
P-HW-48371
M-IY-000105


In [124]:
batch_size = 128
image_size = 128

graph = tf.Graph()
L2_coeff = 0.5e-2
with graph.as_default():
    X_tf = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    y_tf = tf.placeholder(tf.int64, shape=(batch_size, ))
    
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b1 = tf.Variable(tf.zeros([1024]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(X_tf,W1) + b1
    layer1 = tf.nn.relu(logits)
    #layer1 = tf.nn.dropout(layer1, 0.8) # dropout
    
    out = tf.matmul(layer1, W2) + b2
    
    train_prediction = tf.nn.softmax(out)
    
    
    nn        = tf.nn.sparse_softmax_cross_entropy_with_logits(out, y_tf)
    loss      = tf.reduce_mean(nn)
    #loss      = loss + L2_coeff * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)) # L2
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    
    

In [139]:
num_steps = 101

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (y.shape[0] - batch_size)
        batch_data = X[offset:(offset + batch_size), :]
        batch_labels = y[offset:(offset + batch_size)]
        
        feed_dict = {X_tf : batch_data, y_tf : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("\n")
            print("Minibatch loss at step %d: %f" % (step, l))
    res = out.eval(feed_dict={X_tf: X[1300:1428]}, session=session)
    print(res[0].argsort()[:10])
    



Minibatch loss at step 0: 3512.462891


Minibatch loss at step 10: 197623.500000


Minibatch loss at step 20: 71725.664062


Minibatch loss at step 30: 135597.484375


Minibatch loss at step 40: 28076.425781


Minibatch loss at step 50: 27432.785156


Minibatch loss at step 60: 744008.562500


Minibatch loss at step 70: 74678.609375


Minibatch loss at step 80: 77762.578125


Minibatch loss at step 90: 101126.492188


Minibatch loss at step 100: 118138.210938
[1785 1659 3623 1840 1392 1929 5508 2878 4656 3375]


In [141]:
print(accos[1312])
print(res[12].argsort()[:10])

M-0202-06-640-0006
[3623  359 3804 6998  820 1548 2928 1781 1986 3568]


In [164]:
accos[3567]

'M-TS-000074'

In [149]:
X_img = X.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
X_img.shape

(30000, 128, 128, 1)

In [275]:
num_channels = 1
patch_size = 5
depth = 16
num_hidden = 64

graph_con = tf.Graph()
with graph_con.as_default():

  # Input data.
    X_tf_c = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    y_tf_c = tf.placeholder(tf.int64, shape=(batch_size,))
    
    X_tf_test = tf.placeholder(tf.float32, shape=(1, image_size, image_size, num_channels))

  
  # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth * 4 , num_hidden], stddev=0.1)) 
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #hidden1 = tf.nn.dropout(hidden1, 0.9)
        #norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
        #print("hidden1", hidden1.get_shape().as_list())

        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        #norm2 = tf.nn.lrn(hidden2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm2')
        pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #hidden2 = tf.nn.dropout(hidden2, 0.9)
        #print("hidden2", hidden2.get_shape().as_list())

        shape = hidden2.get_shape().as_list()
        reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        #print("reshape", reshape.get_shape().as_list())
        #print("layer3_weights", layer3_weights.get_shape().as_list())
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.matmul(hidden, layer4_weights) + layer4_biases
        return hidden

    # Training computation.
    logits = model(X_tf_c)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_tf_c)
    loss = tf.reduce_mean(cross_entropy, name='cross_entropy')
    
    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    learn_rate  = tf.train.exponential_decay(0.25, global_step, 100, 0.7) # <-- learning rate
    optimizer   = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_step=global_step)
    
    test_prediction = model(X_tf_test)
    

In [277]:
num_steps = 101

with tf.Session(graph=graph_con) as session:
    init_op = tf.initialize_all_variables()
    saver = tf.train.Saver()
    init_op.run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (y.shape[0] - batch_size)
        batch_data = X_img[offset:(offset + batch_size), :]
        batch_labels = y[offset:(offset + batch_size)]
        
        feed_dict = {X_tf_c : batch_data, y_tf_c : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("\n")
            print("Minibatch loss at step %d: %f, rate %f" % (step, l, learn_rate.eval()))
    
    save_path = saver.save(session, "visualsearch_model.ckpt")
    res = test_prediction.eval(feed_dict={X_tf_test: X_img[0:1]}, session=session)
    print(res[0].argsort()[-10:])
    res = test_prediction.eval(feed_dict={X_tf_test: X_img[10:11]}, session=session)
    print(res[0].argsort()[-10:])



Minibatch loss at step 0: 39.867405, rate 0.249110


Minibatch loss at step 10: 9.649906, rate 0.240381


Minibatch loss at step 20: 9.458815, rate 0.231959


Minibatch loss at step 30: 9.345951, rate 0.223831


Minibatch loss at step 40: 9.434006, rate 0.215988


Minibatch loss at step 50: 9.427074, rate 0.208420


Minibatch loss at step 60: 9.592749, rate 0.201117


Minibatch loss at step 70: 9.343151, rate 0.194071


Minibatch loss at step 80: 9.371765, rate 0.187271


Minibatch loss at step 90: 9.415790, rate 0.180709


Minibatch loss at step 100: 9.309351, rate 0.174377
[170 108 119 120  81  51 443 439 469 388]
[170 108 119 120  81  51 443 439 469 388]


In [272]:

test_idx = 305
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session(graph=graph_con) as session:
    init_op = tf.initialize_all_variables()
    saver = tf.train.Saver()
    init_op.run()
    # Restore variables from disk.
    saver.restore(session, "visualsearch_model.ckpt")
    res = test_prediction.eval(feed_dict={X_tf_test: X_img[test_idx:test_idx+1]}, session=session)
    print([accos[uid] for uid in y[test_idx:test_idx+1]])
    print(res[0].argsort()[-20:])
    print(accos[861])

['M-MF-KL-07-82JO']
[299 292 255 342 266 302  22 269 261 329 368 117 346 264 290   0 350 268
 126  38]
M-ER-WSO-CO005B


In [198]:
y[0]

0

In [191]:
accos[2934]

'M-FM-KL-0065'

In [ ]:
[ 465 2367 1646 5952  910 2745 1570 1526  246 1266 3804  501 3485 1824  638
 1735 2709   81  564  861]